# Вступ до інженерії промптів
Інженерія промптів - це процес розробки та оптимізації промптів для завдань обробки природної мови. Він включає вибір правильних промптів, налаштування їхніх параметрів та оцінку їх ефективності. Інженерія промптів є критично важливою для досягнення високої точності та ефективності в моделях NLP. У цьому розділі ми дослідимо основи інженерії промптів, використовуючи моделі OpenAI для експериментів.

### Вправа 1: Токенізація
Дослідіть токенізацію за допомогою tiktoken - швидкого токенізатора з відкритим кодом від OpenAI
Дивіться [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) для додаткових прикладів.


In [26]:
# ВПРАВА:
# 1. Спочатку запустіть вправу як є
# 2. Змініть текст на будь-який промпт, який ви хочете використати, і перезапустіть, щоб побачити токени

import tiktoken

# Визначте промпт, який ви хочете токенізувати
text = f"""
Юпітер - п'ята планета від Сонця і найбільша \
в Сонячній системі. Це газовий гігант з масою \
в одну тисячну від маси Сонця, але в два з половиною \
рази більшою за масу всіх інших планет Сонячної системи разом узятих. \
Юпітер - один з найяскравіших об'єктів, видимих неозброєним оком \
на нічному небі, і був відомий стародавнім цивілізаціям ще \
до писаної історії. Він названий на честь римського бога Юпітера.[19] \
При спостереженні з Землі Юпітер може бути настільки яскравим, що його відбите \
світло може відкидати видимі тіні,[20] і в середньому він є третім за яскравістю \
природним об'єктом на нічному небі після Місяця та Венери.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

[198, 140, 106, 8164, 27385, 58060, 482, 5173, 6, 4329, 44613, 5173, 3114, 7486, 8341, 1506, 5927, 27385, 7094, 28599, 17766, 10589, 4329, 84954, 6850, 19039, 10124, 27385, 29118, 12426, 1506, 5927, 28599, 17766, 4329, 6148, 2156, 27385, 12415, 93099, 27385, 13, 1301, 99, 1532, 25756, 19188, 6856, 44938, 25756, 27385, 15752, 7486, 1830, 11122, 11562, 18437, 1482, 12182, 5927, 53419, 2156, 3865, 11047, 13810, 4329, 6148, 2156, 3865, 5927, 27385, 7094, 11562, 18437, 1840, 28599, 17766, 10589, 4329, 11, 21022, 3114, 1532, 5927, 74254, 1506, 11122, 29619, 6856, 1840, 13999, 12182, 39479, 1840, 14391, 27385, 29118, 12426, 1482, 12182, 44946, 11562, 18437, 3865, 35714, 27385, 10693, 84954, 2156, 30480, 10693, 5173, 3114, 7486, 8341, 28599, 17766, 4329, 6148, 13999, 141, 245, 93099, 1840, 39479, 12507, 14257, 9136, 4329, 23311, 10693, 13, 1301, 106, 8164, 27385, 58060, 482, 53419, 19479, 11122, 6850, 19039, 4329, 66144, 28086, 27385, 30480, 10693, 21923, 6, 141, 242, 4898, 1830, 27385, 5591, 

[b'\n',
 b'\xd0',
 b'\xae',
 b'\xd0\xbf',
 b'\xd1\x96',
 b'\xd1\x82\xd0\xb5\xd1\x80',
 b' -',
 b' \xd0\xbf',
 b"'",
 b'\xd1\x8f',
 b'\xd1\x82\xd0\xb0',
 b' \xd0\xbf',
 b'\xd0\xbb',
 b'\xd0\xb0\xd0\xbd',
 b'\xd0\xb5\xd1\x82',
 b'\xd0\xb0',
 b' \xd0\xb2',
 b'\xd1\x96',
 b'\xd0\xb4',
 b' \xd0\xa1',
 b'\xd0\xbe\xd0\xbd',
 b'\xd1\x86',
 b'\xd1\x8f',
 b' \xd1\x96',
 b' \xd0\xbd',
 b'\xd0\xb0\xd0\xb9',
 b'\xd0\xb1',
 b'\xd1\x96',
 b'\xd0\xbb\xd1\x8c',
 b'\xd1\x88',
 b'\xd0\xb0',
 b' \xd0\xb2',
 b' \xd0\xa1',
 b'\xd0\xbe\xd0\xbd',
 b'\xd1\x8f',
 b'\xd1\x87',
 b'\xd0\xbd',
 b'\xd1\x96',
 b'\xd0\xb9',
 b' \xd1\x81\xd0\xb8\xd1\x81\xd1\x82\xd0\xb5\xd0\xbc',
 b'\xd1\x96',
 b'.',
 b' \xd0',
 b'\xa6',
 b'\xd0\xb5',
 b' \xd0\xb3',
 b'\xd0\xb0\xd0\xb7',
 b'\xd0\xbe\xd0\xb2',
 b'\xd0\xb8\xd0\xb9',
 b' \xd0\xb3',
 b'\xd1\x96',
 b'\xd0\xb3',
 b'\xd0\xb0\xd0\xbd',
 b'\xd1\x82',
 b' \xd0\xb7',
 b' \xd0\xbc',
 b'\xd0\xb0\xd1\x81',
 b'\xd0\xbe',
 b'\xd1\x8e',
 b' \xd0\xb2',
 b' \xd0\xbe\xd0\xb4',
 b'\xd0\xbd'

### Вправа 2: Перевірка налаштування ключа Github Models

Запустіть код нижче, щоб перевірити, чи правильно налаштована кінцева точка Github Models. Код просто випробовує простий базовий промпт і перевіряє завершення. Вхідний текст `о скажи, чи бачиш ти` має завершитись відповідно до контексту.


In [ ]:
pip install azure-ai-inference


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "Ви - корисний асистент.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Виклик допоміжного методу

### 1. Встановіть основний вміст або текст промпту
text = f"""
о скажи, чи бачиш ти
"""

### 2. Використайте його в шаблоні промпту нижче
prompt = f"""
```{text}```
"""

## 3. Запустіть промпт
response = get_completion(prompt, client, model_name)
print(response)

ClientAuthenticationError: (unauthorized) Bad credentials
Code: unauthorized
Message: Bad credentials

### Вправа 3: Вигадки
Дослідіть, що відбувається, коли ви просите LLM повернути завершення для промпту про тему, яка може не існувати, або про теми, про які вона може не знати, оскільки вони були за межами її попередньо навченого набору даних (більш пізні). Подивіться, як змінюється відповідь, якщо ви спробуєте інший промпт або іншу модель.

In [22]:

## Встановіть текст для простого промпту або основного вмісту
## Промпт показує формат шаблону з текстом - додайте підказки, команди тощо, якщо потрібно
## Запустіть завершення
text = f"""
створіть план уроку про Марсіанську війну 2076 року.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

ClientAuthenticationError: (unauthorized) Bad credentials
Code: unauthorized
Message: Bad credentials

### Вправа 4: На основі інструкцій
Використовуйте змінну "text" для встановлення основного вмісту
та змінну "prompt" для надання інструкції, пов'язаної з цим основним вмістом.

Тут ми просимо модель підсумувати текст для учня другого класу

In [ ]:
# Тестовий приклад
# https://platform.openai.com/playground/p/default-summarize

## Приклад тексту
text = f"""
Юпітер - п'ята планета від Сонця і найбільша \
в Сонячній системі. Це газовий гігант з масою \
в одну тисячну від маси Сонця, але в два з половиною \
рази більшою за масу всіх інших планет Сонячної системи разом узятих. \
Юпітер - один з найяскравіших об'єктів, видимих неозброєним оком \
на нічному небі, і був відомий стародавнім цивілізаціям ще \
до писаної історії. Він названий на честь римського бога Юпітера.[19] \
При спостереженні з Землі Юпітер може бути настільки яскравим, що його відбите \
світло може відкидати видимі тіні,[20] і в середньому він є третім за яскравістю \
природним об'єктом на нічному небі після Місяця та Венери.
"""

## Встановіть промпт
prompt = f"""
Підсумуйте наданий вам контент для учня другого класу.
```{text}```
"""

## Запустіть промпт
response = get_completion(prompt, client, model_name)
print(response)

### Вправа 5: Складний промпт
Спробуйте запит, який має системні повідомлення, повідомлення користувача та асистента
Система встановлює контекст асистента
Повідомлення користувача і асистента забезпечують контекст багатокрокової розмови

Зверніть увагу, як особистість асистента встановлена як "саркастична" в системному контексті.
Спробуйте використати інший контекст особистості. Або спробуйте іншу серію повідомлень вводу/виводу

In [ ]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "Ви - саркастичний асистент."},
        {"role": "user", "content": "Хто виграв чемпіонат світу в 2020 році?"},
        {"role": "assistant", "content": "Як ви думаєте, хто виграв? Los Angeles Dodgers, звичайно."},
        {"role": "user", "content": "Де це відбувалося?"}
    ]
)
print(response.choices[0].message.content)

### Вправа: Дослідіть свою інтуїцію
Наведені вище приклади дають вам шаблони, які ви можете використовувати для створення нових промптів (простих, складних, інструкційних тощо) - спробуйте створити інші вправи для дослідження деяких інших ідей, про які ми говорили, таких як приклади, підказки та інше.